In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 200     # number of variables
num_ineq = 200    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_200-200.csv")

  6%|████▌                                                                           | 57/1000 [01:07<23:23,  1.49s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 12%|█████████▍                                                                     | 120/1000 [02:25<17:56,  1.22s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 17%|█████████████▏                                                                 | 167/1000 [03:24<14:52,  1.07s/it]

...

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 70%|███████████████████████████████████████████████████████                        | 697/1000 [14:15<05:49,  1.15s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [20:26<00:00,  1.23s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -46.730136         16.508989      0.332651
std       0.444228          2.188989      0.079426
min     -47.932033          9.702206      0.272455
25%     -47.025423         14.941754      0.293414
50%     -46.752522         16.485873      0.307633
75%     -46.441484         18.010361      0.346934
max     -45.188960         24.531775      1.151797
Number of infeasible solution: 1000


In [7]:
df = pd.read_csv("result/cq_heur_rnd_200-200.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -46.726894         16.542221      0.328783
std      0.469392          1.976129      0.063962
min    -47.811411         11.747271      0.272783
25%    -47.081983         15.132501      0.286565
50%    -46.754793         16.507955      0.296470
75%    -46.360496         17.777287      0.369865
max    -45.698447         21.269113      0.697577
Number of infeasible solution: 100


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 9256.08
Epoch 1, Validation Loss: 5.83
Epoch 2, Validation Loss: -6.76
Epoch 3, Validation Loss: -10.87
Epoch 4, Validation Loss: -14.71
Epoch 5, Validation Loss: -16.20
Epoch 6, Validation Loss: -19.37
Epoch 7, Validation Loss: -19.62
Epoch 8, Validation Loss: -21.89
Epoch 9, Validation Loss: -21.39
Epoch 10, Validation Loss: -22.98
Epoch 11, Validation Loss: -23.98
Epoch 12, Validation Loss: -23.43
Epoch 13, Validation Loss: -24.29
Epoch 14, Validation Loss: -25.07
Epoch 15, Validation Loss: -23.75
Epoch 16, Validation Loss: -24.17
Epoch 17, Validation Loss: -21.96
Epoch 18, Validation Loss: -26.21
Epoch 19, Validation Loss: -27.79
Epoch 20, Validation Loss: -26.54
Epoch 21, Validation Loss: -26.27
Epoch 22, Validation Loss: -26.40
Epoch 23, Validation Loss: -27.66
Epoch 24, Validation Loss: -22.84
Epoch 25, Validation Loss: -29.50
Epoch 26, Validation Loss: -26.94
Epoch 27, Validation Loss: -27.53
Epoch 28, Validation Loss: -28.32
Epoch 29, Validation Loss:

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_200-200.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:01<00:00,  8.23it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -31.209237          0.002694      0.004614
std       0.196035          0.017749      0.002903
min     -31.604665          0.000000      0.001510
25%     -31.380844          0.000000      0.003013
50%     -31.245468          0.000000      0.003568
75%     -31.058942          0.000000      0.005091
max     -30.328813          0.216221      0.022337
Number of infeasible solution: 34


In [8]:
df = pd.read_csv("result/cq_lr_200-200.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -31.205519          0.003159      0.004386
std      0.182342          0.016189      0.002362
min    -31.543232          0.000000      0.001999
25%    -31.360568          0.000000      0.003041
50%    -31.247028          0.000000      0.003819
75%    -31.061275          0.000000      0.004726
max    -30.772018          0.104523      0.018421
Number of infeasible solution: 5


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5004.20
Epoch 1, Validation Loss: -3.16
Epoch 2, Validation Loss: -11.43
Epoch 3, Validation Loss: -17.06
Epoch 4, Validation Loss: -16.42
Epoch 5, Validation Loss: -20.27
Epoch 6, Validation Loss: -20.50
Epoch 7, Validation Loss: -22.01
Epoch 8, Validation Loss: -22.74
Epoch 9, Validation Loss: -22.62
Epoch 10, Validation Loss: -20.28
Epoch 11, Validation Loss: -24.87
Epoch 12, Validation Loss: -23.98
Epoch 13, Validation Loss: -25.92
Epoch 14, Validation Loss: -22.53
Epoch 15, Validation Loss: -26.84
Epoch 16, Validation Loss: -24.07
Epoch 17, Validation Loss: -25.55
Epoch 18, Validation Loss: -23.60
Epoch 19, Validation Loss: -27.41
Epoch 20, Validation Loss: -27.43
Epoch 21, Validation Loss: -27.05
Epoch 22, Validation Loss: -26.44
Epoch 23, Validation Loss: -28.29
Epoch 24, Validation Loss: -18.29
Epoch 25, Validation Loss: -28.14
Epoch 26, Validation Loss: -26.29
Epoch 27, Validation Loss: -27.45
Epoch 28, Validation Loss: -25.93
Epoch 29, Validation Los

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_200-200.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:06<00:00,  7.91it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -31.254160          0.006354      0.004673
std       0.338234          0.043944      0.003048
min     -31.775507          0.000000      0.001505
25%     -31.506309          0.000000      0.003040
50%     -31.313588          0.000000      0.003758
75%     -31.041621          0.000000      0.005000
max     -28.240721          0.584146      0.033856
Number of infeasible solution: 38


In [9]:
df = pd.read_csv("result/cq_lt_200-200.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -31.260533          0.003983      0.004528
std      0.319193          0.039827      0.002572
min    -31.760595          0.000000      0.002011
25%    -31.516941          0.000000      0.003405
50%    -31.319818          0.000000      0.004036
75%    -31.048672          0.000000      0.004585
max    -30.137631          0.398268      0.020412
Number of infeasible solution: 1


### Parametric Learning Then Rounding

In [10]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [12]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [13]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5051.07
Epoch 1, Validation Loss: -7.04
Epoch 2, Validation Loss: -14.87
Epoch 3, Validation Loss: -21.93
Epoch 4, Validation Loss: -25.11
Epoch 5, Validation Loss: -26.60
Epoch 6, Validation Loss: -29.16
Epoch 7, Validation Loss: -29.11
Epoch 8, Validation Loss: -29.42
Epoch 9, Validation Loss: -30.08
Epoch 10, Validation Loss: -29.61
Epoch 11, Validation Loss: -30.43
Epoch 12, Validation Loss: -31.61
Epoch 13, Validation Loss: -31.18
Epoch 14, Validation Loss: -31.79
Epoch 15, Validation Loss: -32.95
Epoch 16, Validation Loss: -32.41
Epoch 17, Validation Loss: -32.69
Epoch 18, Validation Loss: -32.00
Epoch 19, Validation Loss: -32.62
Epoch 20, Validation Loss: -33.67
Epoch 21, Validation Loss: -33.64
Epoch 22, Validation Loss: -33.68
Epoch 23, Validation Loss: -33.38
Epoch 24, Validation Loss: -34.01
Epoch 25, Validation Loss: -33.54
Epoch 26, Validation Loss: -33.87
Epoch 27, Validation Loss: -33.60
Epoch 28, Validation Loss: -33.47
Epoch 29, Validation Los

In [14]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_pr_200-200.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:01<00:00,  8.23it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03       1000.000000   1000.000000
mean  -3.776084e+01          0.535398      0.000840
std    6.753534e-13          0.356764      0.000506
min   -3.776084e+01          0.000000      0.000000
25%   -3.776084e+01          0.256559      0.000513
50%   -3.776084e+01          0.489904      0.001001
75%   -3.776084e+01          0.777848      0.001010
max   -3.776084e+01          1.723091      0.002929
Number of infeasible solution: 975


In [15]:
df = pd.read_csv("result/cq_pr_200-200.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean  -3.776084e+01          0.531382      0.000896
std    4.284734e-14          0.338466      0.000495
min   -3.776084e+01          0.000000      0.000000
25%   -3.776084e+01          0.321623      0.000998
50%   -3.776084e+01          0.488768      0.001002
75%   -3.776084e+01          0.699078      0.001010
max   -3.776084e+01          1.399221      0.002000
Number of infeasible solution: 95
